In [1]:
# from model.model_type import ModelType
from service.site_scraper import SiteScraper

gsp_scraper = SiteScraper(
    name="gsp",
    base_url="https://www.gsp.ro",
    traffic=3.6,
    time_selector="p.data-author",
    block_selector="div.news-item",
    link_selector="h2 a",
    title_strategy="text"
)

digisport_scraper = SiteScraper(
    name="digisport",
    base_url="https://www.digisport.ro",
    traffic=3.3,
    time_selector="cite",
    block_selector="article.article",
    link_selector="a.article-link, h3 a.widget-latest-list-item-link",
    title_strategy="attribute",
    title_attribute="title",
    # model=ModelType.SPACY
)

fanatik_scraper = SiteScraper(
    name="fanatik",
    base_url="https://www.fanatik.ro",
    traffic=2.9,  # Estimated, adjust as needed
    time_selector="span.date",  # Often used near article blocks
    block_selector="div.article",  # Container for each article
    link_selector="h3 a",  # Headline links
    title_strategy="text",
    # model=ModelType.GPT
)

prosport_scraper = SiteScraper(
    name="prosport",
    base_url="https://www.prosport.ro",
    traffic=3.1,
    time_selector="span.date",
    block_selector="h2.article__title",  # Directly targets the title block
    link_selector="a",  # The anchor inside the h2
    title_strategy="text",
    # model=ModelType.CLAUDE
)

sites = [gsp_scraper, digisport_scraper, fanatik_scraper, prosport_scraper]
# sites = [prosport_scraper]
total_traffic = sum(site.traffic for site in sites)

In [2]:
for site in sites:
    site.compute_weight(total_traffic)
    site.scrape_recent_articles(minutes=360)
    # site.save_to_csv()
    # site.short_print()


In [3]:
for site in sites:
    site.save_to_csv()
    # site.short_print()

In [4]:
from service.story_clusterer import StoryClusterer

clusterer = StoryClusterer(sites, 360, 0.3, 0.2)
clusterer.cluster_stories()

clusterer.print_matched_clusters()


🧠 Cluster #1 — Score: 0.535

🧠 Cluster #2 — Score: 0.504

🧠 Cluster #3 — Score: 0.496

🧠 Cluster #4 — Score: 0.496

🔍 Matched Clusters Across Multiple Sites

🧠 Cluster #1 — Score: 0.535 — Sites: gsp, fanatik
------------------------------------------------------------
📰 Schimbare de ultimă oră la echipa națională! Un fotbalist de la Rapid a fost convocat de urgență
🔗 https://www.fanatik.ro/schimbare-de-ultima-ora-la-echipa-nationala-un-fotbalist-de-la-rapid-a-fost-convocat-de-urgenta-21289843
🔑 Keywords: Modificare, accidentat grav, jucător, locul, lotul, ratează convocarea
🏷️ Entities: FCSB, Universitatea Craiova, echipa rivală, naționalei României
------------------------------------------------------------
📰 OUT de la națională! Un fotbalist de la Rapid îi ia locul
🔗 https://www.gsp.ro/fotbal/nationala/rares-pop-inlocuitor-mihnea-radulescu-romania-u21-851150.html
🔑 Keywords: accidentat grav, derby, fotbalistul, înlocuiește
🏷️ Entities: Arena Națională, FCSB, Federația Română de Fot

In [5]:

top_stories = clusterer.score_clusters()

for i, story in enumerate(top_stories[:5], 1):
    print(f"\n🏆 Top {i} — Score: {story['score']}")
    for article in story["articles"]:
        print(f"- {article.site}: {article.title}")



🏆 Top 1 — Score: 0.535
- gsp: FRF a făcut calculele: „Cum se califică România direct la Campionatul Mondial”
- digisport: FRF a făcut toate calculele! Cum poate ajunge România la Campionatul Mondial

🏆 Top 2 — Score: 0.504
- fanatik: Schimbare de ultimă oră la echipa națională! Un fotbalist de la Rapid a fost convocat de urgență
- gsp: OUT de la națională! Un fotbalist de la Rapid îi ia locul

🏆 Top 3 — Score: 0.496
- digisport: Blănuță a spus ”STOP”: ”S-a gândit serios să plece imediat de la Dinamo Kiev”. Ce a urmat
- prosport: Vladislav Blănuță a vrut să plece de la Dinamo Kiev. De ce nu a putut!

🏆 Top 4 — Score: 0.496
- digisport: 200.000.000 de euro pentru a vinde FCSB? Gigi Becali a făcut anunțul
- prosport: Ce s-a ales de fotbalistul pentru care Gigi Becali a pierdut aproape un milion de euro
